In [1]:
import os
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 2s (114 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to pro

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_revised_data = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_revised_data.createOrReplaceTempView('home_sales')
spark.sql("SELECT * FROM home_sales LIMIT 10").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
  SELECT bedrooms, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 4
  GROUP BY bedrooms""").show()

+--------+---------+
|bedrooms|avg_price|
+--------+---------+
|       4|299661.01|
+--------+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built, that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
  SELECT date_built, AVG(bedrooms) AS bedrooms, AVG(bathrooms) AS bathrooms, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY date_built
  ORDER BY date_built ASC""").show()

+----------+--------+---------+---------+
|date_built|bedrooms|bathrooms|avg_price|
+----------+--------+---------+---------+
|      2010|     3.0|      3.0|292859.62|
|      2011|     3.0|      3.0|291117.47|
|      2012|     3.0|      3.0|293683.19|
|      2013|     3.0|      3.0|295962.27|
|      2014|     3.0|      3.0|290852.27|
|      2015|     3.0|      3.0| 288770.3|
|      2016|     3.0|      3.0|290555.07|
|      2017|     3.0|      3.0|292676.79|
+----------+--------+---------+---------+



In [7]:
# 5. What is the average price of a home for each year the home was built, that have 3 bedrooms, 3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
  SELECT date_built, AVG(bedrooms) AS bedrooms, AVG(bathrooms) AS bathrooms, AVG(floors) AS floors, MIN(sqft_living) AS min_sqft_living , ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY date_built
  ORDER BY date_built ASC""").show()

+----------+--------+---------+------+---------------+---------+
|date_built|bedrooms|bathrooms|floors|min_sqft_living|avg_price|
+----------+--------+---------+------+---------------+---------+
|      2010|     3.0|      3.0|   2.0|           2002|285010.22|
|      2011|     3.0|      3.0|   2.0|           2002|276553.81|
|      2012|     3.0|      3.0|   2.0|           2009|307539.97|
|      2013|     3.0|      3.0|   2.0|           2003|303676.79|
|      2014|     3.0|      3.0|   2.0|           2000|298264.72|
|      2015|     3.0|      3.0|   2.0|           2002|297609.97|
|      2016|     3.0|      3.0|   2.0|           2001| 293965.1|
|      2017|     3.0|      3.0|   2.0|           2000|280317.58|
+----------+--------+---------+------+---------------+---------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places, having an average home price greater than or equal to $350,000? Order by descending view rating.
# Determine the run time for this query.

start_time = time.time()

spark.sql("""
  SELECT view, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  GROUP BY view
  HAVING AVG(price) >= 350000
  ORDER BY view DESC;""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.1329290866851807 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached("home_sales")

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates the average price of a home per "view" rating, rounded to two decimal places, having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
  SELECT view, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  GROUP BY view
  HAVING AVG(price) >= 350000
  ORDER BY view DESC;""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.5212717056274414 seconds ---


Comparing the query that calculates the average home price per 'view' rating (rounded to two decimal places) and filters for an average price of $350,000 or more, it is evident that caching the `home_sales` data significantly reduces runtime compared to when the data is not cached.

In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_revised_data.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_parquet")

In [13]:
# 11. Read the parquet formatted data.
home_sales_partitioned = spark.read.parquet("home_sales_parquet")

In [14]:
# 12. Create a temporary table for the parquet data.
home_sales_partitioned.createOrReplaceTempView("home_sales_p")

In [15]:
# 13. Using the parquet DataFrame, run the query that calculates the average price of a home per "view" rating, rounded to two decimal places, having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
  SELECT view, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales_p
  GROUP BY view
  HAVING AVG(price) >= 350000
  ORDER BY view DESC;""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.1621782779693604 seconds ---


The runtime results indicate that the query performance, which calculates the average home price per 'view' rating (rounded to two decimal places) for homes with an average price of $350,000 or more, is significantly worse when the `home_sales` data is partitioned by `date_built` in a Parquet file compared to when the data is cached.

In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False